In [1]:

# coding: utf-8

# In[1]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')


# In[2]:


import findspark


# In[33]:


import pyspark


# In[34]:


from pyspark import SparkContext


# In[35]:


from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('adult').getOrCreate()
data_train = spark.read.csv(r'/home/ubuntu/train.csv', inferSchema = True, header=True)
pd.DataFrame(data_train.take(20), columns = data_train.columns)


# In[38]:


data_train =pd.read_csv(r'/home/ubuntu/train.csv')
data_train.shape


# In[194]:


print(data_train.columns)


# In[40]:


data_train.head()


# In[41]:


data_train.info()


# In[42]:


data_train.describe()


# In[43]:


import seaborn as sns


# In[44]:


numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea,list(data_train.columns)))


# In[45]:


numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea,list(data_train.columns)))
#过滤数值型类别特征
def get_numerical_serial_fea(data,feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique()
        if temp <= 10:
            numerical_noserial_fea.append(fea)
            continue
        numerical_serial_fea.append(fea)
    return numerical_serial_fea,numerical_noserial_fea
numerical_serial_fea,numerical_noserial_fea = get_numerical_serial_fea(data_train,numerical_fea)


# In[46]:


f = pd.melt(data_train, value_vars=numerical_serial_fea)
g = sns.FacetGrid(f, col="variable",  col_wrap=2, sharex=False, sharey=False)
g = g.map(sns.distplot, "value")


# In[47]:


#Ploting Transaction Amount Values Distribution
plt.figure(figsize=(16,12))
plt.suptitle('Transaction Values Distribution', fontsize=22)
plt.subplot(221)
sub_plot_1 = sns.distplot(data_train['loanAmnt'])
sub_plot_1.set_title("loanAmnt Distribuition", fontsize=18)
sub_plot_1.set_xlabel("")
sub_plot_1.set_ylabel("Probability", fontsize=15)

plt.subplot(222)
sub_plot_2 = sns.distplot(np.log(data_train['loanAmnt']))
sub_plot_2.set_title("loanAmnt (Log) Distribuition", fontsize=18)
sub_plot_2.set_xlabel("")
sub_plot_2.set_ylabel("Probability", fontsize=15)


# In[48]:


plt.figure(figsize=(8, 8))
sns.barplot(data_train["employmentLength"].value_counts(dropna=False)[:20],
            data_train["employmentLength"].value_counts(dropna=False).keys()[:20])
plt.show()


# In[49]:


train_loan_fr = data_train.loc[data_train['isDefault'] == 1]
train_loan_nofr = data_train.loc[data_train['isDefault'] == 0]


# In[50]:


fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 8))
train_loan_fr.groupby('grade')['grade'].count().plot(kind='barh', ax=ax1, title='Count of grade fraud')
train_loan_nofr.groupby('grade')['grade'].count().plot(kind='barh', ax=ax2, title='Count of grade non-fraud')
train_loan_fr.groupby('employmentLength')['employmentLength'].count().plot(kind='barh', ax=ax3, title='Count of employmentLength fraud')
train_loan_nofr.groupby('employmentLength')['employmentLength'].count().plot(kind='barh', ax=ax4, title='Count of employmentLength non-fraud')
plt.show()


# In[51]:


#查看缺失值情况
data_train.isnull().sum()


# In[52]:


data_train.info()


# In[146]:


data_train =pd.read_csv(r'/home/ubuntu/train1.csv')
data_train.shape


# In[53]:


print(data_train.columns)


# In[31]:


#查看缺失值情况
data_train.isnull().sum()


# In[54]:


data_train=data_train.drop('employmentLength',axis=1)


# In[55]:


numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea,list(data_train.columns)))

def get_numerical_serial_fea(data,feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique()
        if temp <= 10:
            numerical_noserial_fea.append(fea)
            continue
        numerical_serial_fea.append(fea)
    return numerical_serial_fea,numerical_noserial_fea
numerical_serial_fea,numerical_noserial_fea = get_numerical_serial_fea(data_train,numerical_fea)

data_train[numerical_fea] = data_train[numerical_fea].fillna(data_train[numerical_fea].median())
data_train[category_fea] = data_train[category_fea].fillna(data_train[category_fea].mode())


# In[56]:


data_train.isnull().sum()


# In[57]:


data_train['installment']=abs(data_train['installment'])


# In[58]:


data_train['installment'].head()


# In[59]:


print(data_train.columns)


# In[60]:


data_train.head()


# In[61]:


data_train.isnull().sum()


# In[51]:


data_train.shape


# In[62]:


data_train.info()


# In[63]:


#label-encode:subGrade,postCode,title
for col in tqdm(['employmentTitle', 'postCode', 'title','subGrade']):
    le = LabelEncoder()
    le.fit(list(data_train[col].astype(str).values) )
    data_train[col] = le.transform(list(data_train[col].astype(str).values))
print('Label Encoding ')


# In[56]:





# In[64]:


data_numeric = data_train[numerical_fea]
correlation = data_numeric.corr()

f , ax = plt.subplots(figsize = (7, 7))
plt.title('Correlation of Numeric Features with Price',y=1,size=16)
sns.heatmap(correlation,square = True,  vmax=0.8)


# In[65]:



for data in [data_train]:
    data.drop(['issueDate','id'], axis=1,inplace=True)


# In[66]:


total = len(data_train)
total_amt = data_train.groupby(['isDefault'])['loanAmnt'].sum().sum()
plt.figure(figsize=(12,5))
plt.subplot(121)
plot_tr = sns.countplot(x='isDefault',data=data_train)#data_train‘isDefault’
plot_tr.set_title("Fraud Loan Distribution \n 0: good user | 1: bad user", fontsize=14)
plot_tr.set_xlabel("Is fraud by count", fontsize=16)
plot_tr.set_ylabel('Count', fontsize=16)
for p in plot_tr.patches:
    height = p.get_height()
    plot_tr.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=15) 
    
percent_amt = (data_train.groupby(['isDefault'])['loanAmnt'].sum())
percent_amt = percent_amt.reset_index()
plt.subplot(122)
plot_tr_2 = sns.barplot(x='isDefault', y='loanAmnt',  dodge=True, data=percent_amt)
plot_tr_2.set_title("Total Amount in loanAmnt  \n 0: good user | 1: bad user", fontsize=14)
plot_tr_2.set_xlabel("Is fraud by percent", fontsize=16)
plot_tr_2.set_ylabel('Total Loan Amount Scalar', fontsize=16)
for p in plot_tr_2.patches:
    height = p.get_height()
    plot_tr_2.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total_amt * 100),
            ha="center", fontsize=15)     


# In[68]:


from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import joblib
import pickle
import numpy as np


# In[69]:


data_train=data_train.drop('postCode',axis=1)


# In[70]:


data_train=data_train.drop('title',axis=1)


# In[71]:


data_train=data_train.drop('subGrade',axis=1)


# In[72]:


data_train=data_train.drop('grade',axis=1)


# In[75]:


data_train=data_train.drop('earliesCreditLine',axis=1)


# In[73]:


data_train=data_train.drop('policyCode',axis=1)


# In[76]:


from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb
import joblib
import pickle
import numpy as np


# In[83]:


features = [f for f in data_train.columns if f not in ['id','issueDate','isDefault','postCode','grade','title','subGrade'] ]
x = data_train[features]
y=data_train["isDefault"]


# In[84]:


data_train.info()


# In[85]:


x.columns


# In[86]:


data_train.head()


# In[87]:


x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=3, train_size=0.7)


# In[92]:


from pyspark.ml.classification import RandomForestClassifier
#from sklearn.ensemble import RandomForestClassifier
#rf = RandomForestClassifier()


# In[93]:


rf.fit(x_train, y_train)


# In[94]:


#模型特征输出
importance_value = rf.feature_importances_
importance_var = list(x_train.columns)
importance_var_value = []
for i in range(len(importance_var)):
    importance_var_value.append([importance_var[i], importance_value[i]])
print(importance_var_value)
importance_df = pd.DataFrame()
importance_df = importance_df.append(importance_var_value)


# In[178]:


y_pred = rf.predict(x_test)
from sklearn.metrics import classification_report
import pickle
print (classification_report(y_test,y_pred,target_names=['class_0','class_1'],digits=4)) 

result = np.column_stack([y_pred,y_test])
result = pd.DataFrame(result,columns=['pred','y_test'])
pd.crosstab(result.pred,result.y_test,margins=True)


# In[99]:


x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2, train_size=0.8)


# In[100]:


rf.fit(x_train, y_train)


# In[97]:


importance_value = rf.feature_importances_
importance_var = list(x_train.columns)
importance_var_value = []
for i in range(len(importance_var)):
    importance_var_value.append([importance_var[i], importance_value[i]])
print(importance_var_value)
importance_df = pd.DataFrame()
importance_df = importance_df.append(importance_var_value)


# In[101]:


y_pred = rf.predict(x_test)
from sklearn.metrics import classification_report
import pickle
print (classification_report(y_test,y_pred,target_names=['class_0','class_1'],digits=4)) 

result = np.column_stack([y_pred,y_test])
result = pd.DataFrame(result,columns=['pred','y_test'])
pd.crosstab(result.pred,result.y_test,margins=True)


# In[183]:


y_pred


# In[187]:


from sklearn.linear_model import LogisticRegression


# In[188]:


lr=LogisticRegression()


# In[189]:


rf.fit(x_train, y_train)


# In[191]:


y_pred = rf.predict(x_test)
from sklearn.metrics import classification_report
import pickle
print (classification_report(y_test,y_pred,target_names=['class_0','class_1'],digits=4)) 

result = np.column_stack([y_pred,y_test])
result = pd.DataFrame(result,columns=['pred','y_test'])
pd.crosstab(result.pred,result.y_test,margins=True)


# In[ ]:


#预测新数据的时候，把预测数据代入，然后把截图的代码跑下


# In[231]:


y_yuce=rf.predict(x)


# In[232]:


y_yuce



/home/ubuntu/.local/lib/python3.5/site-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)
/home/ubuntu/.local/lib/python3.5/site-packages/pyspark/context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python

MemoryError: 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46632)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.5/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
